In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import zipfile
pd.set_option("display.max_columns", None)
#local_zip = 'Datasets/02-14-2018.zip'
#zip_ref = zipfile.ZipFile(local_zip,'r' )
#zip_ref.extractall('Datasets/archive-IDS')
#zip_ref.close()


In [ ]:
df = pd.read_csv('../input/ids-intrusion-csv/02-14-2018.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["Label"].value_counts()

In [ ]:
#_=df["Label"].value_counts().plot.bar(rot =0)
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
import plotly.express as px
init_notebook_mode(connected=True)
import plotly.graph_objs as go

fig = go.Figure(data=[
    go.Bar(name='Benign', 
           y=df["Label"].value_counts().values[0:1],
           x=['Benign'],
           text = df["Label"].value_counts()[0:1],
           orientation='v',
           textposition='outside',),
    go.Bar(name='FTP-BruteForce', 
           y=df["Label"].value_counts().values[1:2],
           x=['FTP-BruteForce'],
           text = df["Label"].value_counts()[1:2],
           orientation='v',
           textposition='outside',),
    go.Bar(name='SSH-Bruteforce', 
           y=df["Label"].value_counts().values[2:],
           x=['SSH-Bruteforce'],
           text = df["Label"].value_counts()[2:],
           orientation='v',
           textposition='outside',)
])
# Change the bar mode
fig.update_layout(
                  width=800,
                  height=600,
                  title=f'Class Distribution',
                  yaxis_title='Number of attacks',
                  xaxis_title='Attack Name',)
iplot(fig)

In [ ]:
## shortening the data
df1 = df[df["Label"] == "Benign"][:5000]
df2 = df[df["Label"] == "FTP-BruteForce"][:5000]
df3 = df[df["Label"] == "SSH-Bruteforce"][:5000]

In [ ]:
#df = pd.concat([df1, df2,df3], axis =0)
df = pd.concat([ df1,df2], axis =0)

In [ ]:
df

In [ ]:
#_=df["Label"].value_counts().plot.bar(rot =0)
fig = go.Figure(data=[
    go.Bar(name='Benign', 
           y=df["Label"].value_counts().values[0:1],
           x=['Benign'],
           text = df["Label"].value_counts()[0:1],
           orientation='v',
           textposition='outside',),
    go.Bar(name='FTP-BruteForce', 
           y=df["Label"].value_counts().values[1:2],
           x=['FTP-BruteForce'],
           text = df["Label"].value_counts()[1:2],
           orientation='v',
           textposition='outside',),

])
# Change the bar mode
fig.update_layout(
                  width=800,
                  height=600,
                  title=f'Class Distribution',
                  yaxis_title='Number of attacks',
                  xaxis_title='Attack Name',)
iplot(fig)

In [ ]:
data = df.dropna()
target = data["Label"]
data = data.drop(columns = "Label", axis = 1)


data = data.loc[:, (df != 0).any(axis=0)]

In [ ]:
target =pd.get_dummies(target)


In [ ]:
cor_B = data.corrwith(other = target["Benign"],method = 'pearson')
cor_F = data.corrwith(other = target["FTP-BruteForce"],method = 'pearson')
#cor_S = data.corrwith(other = target["SSH-Bruteforce"],method = 'pearson')

In [ ]:
cor_B.sort_values(ascending= False)

In [ ]:
cor_F.sort_values(ascending= False)

In [ ]:
#cor_S.sort_values(ascending= False)

In [ ]:
data = data.drop(columns = ["Protocol","PSH Flag Cnt","Init Fwd Win Byts","Flow Byts/s","Flow Pkts/s"], axis = 1) 

from sklearn.compose import make_column_selector as selector
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

In [ ]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
model = make_pipeline(preprocessor, LogisticRegression(max_iter = 500))

from sklearn import set_config
set_config(display='diagram') #interactive diagram
model

In [ ]:
from sklearn.model_selection import cross_validate
target = target.drop(["FTP-BruteForce"], axis =1) #avoiding dummy variable trap
cv_results = cross_validate(model, data, target, cv=5)
cv_results

In [ ]:
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "f"{scores.mean():.3f} +/- {scores.std():.3f}")


In [ ]:
data

In [ ]:
# from sklearn.model_selection import train_test_split

# Let's start by splitting our data into train and test sets.
# data_train, data_test, target_train, target_test = train_test_split(data, target, random_state=42)

# _ = model.fit(data_train, target_train)
# data_test.head()
# model.predict(data_test)[:5]
# target_test[:5]


In [ ]:
pp_df = pd.concat([data, target], axis = 1)

In [ ]:
import plotly.express as px
# Import the necessaries libraries
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
pyo.init_notebook_mode()
fig =px.scatter(pp_df, x = "Bwd Pkts/s",y="Fwd Seg Size Min", color = "Benign" )

fig.show()


In [ ]:
sns.scatterplot(data = pp_df, x = "Bwd Pkts/s",y="Fwd Seg Size Min", hue = "Benign" )
xline = np.linspace(pp_df["Bwd Pkts/s"].min(),pp_df["Bwd Pkts/s"].max(), num = 300)
yline = np.linspace(pp_df["Fwd Seg Size Min"].max(), pp_df["Fwd Seg Size Min"].min(), num = 300)
_=plt.plot(xline,yline-9, color = "green")